# Introduction
This Recommender System recommends similar hostels with an explanation and as a result, such recommendations become more effective and more persuasive.

Since there is no data available for hostels in Ireland on data libraries so I have scrapped the data from Hostel World website for the experiment.

Let's start by importing the necessary libraries.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import euclidean_distances

import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Loading and displaying the dataset
df_hostels = pd.read_csv("../input/hybrid.csv", encoding='latin1')

df_hostels.head()

,hostel.id,summary.score,rating.band,Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,...,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Name,Distance,City,Price
0,1,9.1,Superb,9.0,9.6,9.1,9.1,8.6,9.3,8.9,...,0,0,0,0,0,1,Jacobs Inn,0.6km from city centre,Dublin,19.05
1,2,8.7,Fabulous,8.3,8.9,9.7,8.9,8.2,8.2,8.3,...,0,1,1,1,1,1,Abbey Court,0.4km from city centre,Dublin,12.49
2,3,7.9,Very Good,7.6,8.2,8.7,8.3,7.8,7.6,7.4,...,0,0,1,1,1,0,Isaacs Hostel,0.5km from city centre,Dublin,12.00
3,4,8.0,Fabulous,7.7,8.6,9.0,8.5,7.3,7.3,7.3,...,0,0,0,0,0,0,Abrahams Hostel,0.4km from city centre,Dublin,11.00
4,5,7.8,Very Good,7.2,8.1,9.3,8.1,7.3,7.5,7.2,...,0,0,1,1,1,0,Kinlay House Dublin,1km from city centre,Dublin,12.00


Here for some entertainment features, the values are give as 1 and 0. It means 0 = No and 1 = Yes.

# 1. Exploratory Data Analysis

In [ ]:
# General Information
df_hostels.info()

In [ ]:
# Statistical characteristics of numerical features
df_hostels.describe()

Let's draw histograms for some relavent fields:

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplot(221)
plt.hist(df_hostels['Price'].values, bins=20)
plt.title('Price')
plt.subplot(222)
plt.hist(df_hostels['summary.score'].values, bins=20)
plt.title('summary.score')

Most hostels in the dataset are between 10 to 50 Euros.

# 2. Data Preprocessing
Handling Missing Values

In [ ]:
df_hostels.isnull().sum()

No missing value are found.

Now, I'm deleting summary.score, name, and rating.band columns because summary.score is giving the average value for columns like value.for.money, security, location, staff, atmosphere, cleanliness, and facilities. But since we are taking these features separately for similarity computation so we have removed the summary.score column. We can access the name later so name column is removed. The third column rating.band is also not necessary and is removed because it is giving rating based on summary.score i-e if summary.score is between 1 to 3, then rating.band is "Good".

In [3]:
df_hostels.drop(['summary.score', 'Name', 'rating.band'], inplace=True, axis=1);

In [4]:
# Label Encoding
le = LabelEncoder()
df_hostels['Distance'] = le.fit_transform(df_hostels['Distance'])
df_hostels['City'] = le.fit_transform(df_hostels['City'])

df_hostels.head(3)

,hostel.id,Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,Board.Games,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Distance,City,Price
0,1,9.0,9.6,9.1,9.1,8.6,9.3,8.9,1,0,0,0,0,0,1,5,4,19.05
1,2,8.3,8.9,9.7,8.9,8.2,8.2,8.3,1,0,1,1,1,1,1,3,4,12.49
2,3,7.6,8.2,8.7,8.3,7.8,7.6,7.4,1,0,0,1,1,1,0,4,4,12.00


# 3. Modeling

In [5]:
# Function get all cities
def getSameCityRows(anchor_id):
    getRow = df_hostels.loc[anchor_id-1, :]
    city = getRow['City']
    df_sorted = df_hostels.loc[df_hostels['City'] == int(city)]
    return df_sorted

def get_recommendations11(df, anchor_id):
    featureString = 'Distance,Price,City'
    getRow = df.loc[anchor_id-1, :]
    
    featureString = 'Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,Board.Games,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Distance,City,Price'
    
    features = featureString.split(',')
    
    df_sorted = df.copy()
    df_sorted = pd.concat([df_sorted[df_sorted['hostel.id'] == anchor_id],df_sorted[df_sorted['hostel.id'] != anchor_id]])
    df_features = df_sorted[features].copy()
    df_features = normalize_features(df_features)
    
    # compute the distances
    X = df_features.values
    Y = df_features.values[0].reshape(1, -1)
    distances = euclidean_distances(X, Y)
    df_sorted['similarity_distance'] = distances
    return df_sorted.sort_values('similarity_distance').reset_index(drop=True)
def get_recommendations(df, anchor_id):
    featureString = 'Distance,Price,City'
    getRow = df.loc[anchor_id-1, :]
    
    if(getRow['Value.for.money'] != 0.0):
        featureString+=",Value.for.money"
    if(getRow['Security'] != 0.0):
        featureString+=",Security"
    if(getRow['Location'] != 0.0):
        featureString+=",Location"
    if(getRow['Staff'] != 0.0):
        featureString+=",Staff"
    if(getRow['Atmosphere'] != 0.0):
        featureString+=",Atmosphere"
    if(getRow['Cleanliness'] != 0.0):
        featureString+=",Cleanliness"
    if(getRow['Facilities'] != 0.0):
        featureString+=",Facilities"
        
    if(getRow['Board.Games'] != 0):
        featureString+=",Board.Games"
    if(getRow['Dvds'] != 0):
        featureString+=",Dvds"
    if(getRow['Foosball'] != 0):
        featureString+=",Foosball"
    if(getRow['Games.Room'] != 0):
        featureString+=",Games.Room"
    if(getRow['PlayStation'] != 0):
        featureString+=",PlayStation"
    if(getRow['Pool.Table'] != 0):
        featureString+=",Pool.Table"
    if(getRow['Wifi'] != 0):
        featureString+=",Wifi"
    
    features = featureString.split(',')
    
    df_sorted = df.copy()
    df_sorted = pd.concat([df_sorted[df_sorted['hostel.id'] == anchor_id],df_sorted[df_sorted['hostel.id'] != anchor_id]])
    df_features = df_sorted[features].copy()
    df_features = normalize_features(df_features)
    
    # compute the distances
    X = df_features.values
    Y = df_features.values[0].reshape(1, -1)
    distances = euclidean_distances(X, Y)
    df_sorted['similarity_distance'] = distances
    return df_sorted.sort_values('similarity_distance').reset_index(drop=True)
    
def normalize_features(df):
    for col in df.columns:
        # fill any NaN's with the mean
        df[col] = df[col].fillna(df[col].mean())
        df[col] = StandardScaler().fit_transform(df[col].values.reshape(-1, 1))
    return df

def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list    

# 4. Explainable Processing Methods

Now, I will code to put an explanation before the hostels' recommendations according to this logic.

If all three recommended hostels have 8 or more than 8 rating in any of the following columns (Value.for.money, Security, Location, Staff, Atmosphere, Cleanliness, Facilities), then mention those in the explanation.

Also, If all three recommended hostels have 1 which means yes in any of the following columns (Board.Games, Dvds, Foosball, Games.Room, PlayStation, Pool.Table or Wifi), then mention those in the explanation.

In [6]:
def processSummary(df):
    tf = (df.values > 7.9)
    if False in tf[:]:
        return False
    else:
        return True
def processEntertainment(df):
    tf = (df.values == 1)
    if False in tf[:]:
        return False
    else:
        return True
def getExplaination(finalShowDown):
    # Summary Fields
    value_for_money = processSummary(finalShowDown['Value.for.money'])
    security = processSummary(finalShowDown['Security'])
    staff = processSummary(finalShowDown['Staff'])
    atmosphere = processSummary(finalShowDown['Atmosphere'])
    clean = processSummary(finalShowDown['Cleanliness'])
    facilities = processSummary(finalShowDown['Facilities'])
    location = processSummary(finalShowDown['Location'])
    
    summary = ""
    finalDecision = "Similar hostels "
    if(value_for_money):
        summary+='Value for money, '
    if(security):
        summary+='Security, '
    if(staff):
        summary+='Staff, '
    if(atmosphere):
        summary+='Atmosphere, '
    if(clean):
        summary+='Cleanliness, '
    if(facilities):
        summary+='Facilities, '
    if(location):
        summary+='Location, '

    if summary != "":
        summary = removeLastOccurence(summary, ",")
        summary_split = summary.split(",")
        if(len(summary_split) > 1):
            summary = removeAgainOccurence(summary, ",")
        finalDecision+="who are famous for excellent "+summary
        
    # Entertainment Fields
    board_games = processEntertainment(finalShowDown['Board.Games'])
    dvd = processEntertainment(finalShowDown['Dvds'])
    foosball = processEntertainment(finalShowDown['Foosball'])
    games_room = processEntertainment(finalShowDown['Games.Room'])
    play_station = processEntertainment(finalShowDown['PlayStation'])
    pool_table = processEntertainment(finalShowDown['Pool.Table'])
    wifi = processEntertainment(finalShowDown['Wifi'])
    
    entertainment = ""
    if(board_games):
        entertainment+='Board Games, '
    if(dvd):
        entertainment+='DVDs, '
    if(foosball):
        entertainment+='Foosball, '
    if(games_room):
        entertainment+='Games Room, '
    if(play_station):
        entertainment+='PlayStation, '
    if(pool_table):
        entertainment+='Pool Table, '
    if(wifi):
        entertainment+='Wifi, '

    if entertainment != "":
        entertainment = removeLastOccurence(entertainment, ",")
        ent_split = entertainment.split(",")
        if(len(ent_split) > 1):
            entertainment = removeAgainOccurence(entertainment, ",")
        if summary != "":
            finalDecision+="and also have "+entertainment+"."
        else:
            finalDecision+="who have "+entertainment+"."
    else:
        finalDecision+="."
    return removeLastOccurence(finalDecision, " ")

def removeLastOccurence(str_val, delimiter):
    k = str_val.rfind(delimiter)
    new_string = str_val[:k] + "" + str_val[k+1:]
    return new_string
def removeAgainOccurence(str_val, delimiter):
    k = str_val.rfind(delimiter)
    new_string = str_val[:k] + " and" + str_val[k+1:]
    return new_string

Results for all hostel ids using loop

In [15]:
for anchor_id in range(1,121):
    getRows = getSameCityRows(anchor_id)

    recommendations = get_recommendations(getRows, anchor_id)

    finalShowDown = recommendations.head(n=4)

    print("\n") 
    print('\033[1m'+"Test Resuts for hostel.id="+str(anchor_id)+"\n")
    finalOutput = finalShowDown.loc[[1,2,3]]
    explaination = getExplaination(finalOutput) 
    if(explaination == 'Similar hostels.'):
        explaination = 'Similar hostels which have similar Price and have same Distance from the City Centre.' 
    print(explaination)

    display(finalShowDown)



Test Resuts for hostel.id=1

Similar hostels who are famous for excellent Value for money, Security, Staff, Atmosphere, Cleanliness, Facilities and Location and also have Board Games and Wifi.


,hostel.id,Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,Board.Games,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Distance,City,Price,similarity_distance
0,1,9.0,9.6,9.1,9.1,8.6,9.3,8.9,1,0,0,0,0,0,1,5,4,19.05,0.000000
1,9,8.7,9.2,9.6,8.8,8.5,9.2,8.6,1,0,0,0,0,0,1,3,4,16.00,0.319763
2,10,8.6,9.1,8.0,9.3,8.2,9.0,8.7,1,0,1,1,0,1,1,7,4,18.00,0.426986
3,2,8.3,8.9,9.7,8.9,8.2,8.2,8.3,1,0,1,1,1,1,1,3,4,12.49,0.583878




Test Resuts for hostel.id=2

Similar hostels who are famous for excellent Security, Staff and Location and also have Games Room and Pool Table.


,hostel.id,Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,Board.Games,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Distance,City,Price,similarity_distance
0,2,8.3,8.9,9.7,8.9,8.2,8.2,8.3,1,0,1,1,1,1,1,3,4,12.49,0.000000
1,10,8.6,9.1,8.0,9.3,8.2,9.0,8.7,1,0,1,1,0,1,1,7,4,18.00,4.149088
2,3,7.6,8.2,8.7,8.3,7.8,7.6,7.4,1,0,0,1,1,1,0,4,4,12.00,4.157995
3,8,7.8,8.9,8.5,8.6,8.2,7.8,8.1,0,0,1,1,0,1,1,10,4,13.30,4.642671




Test Resuts for hostel.id=3

Similar hostels who are famous for excellent Security, Staff and Location and also have Board Games, Games Room and Pool Table.


...

,hostel.id,Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,Board.Games,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Distance,City,Price,similarity_distance
0,118,9.4,9.7,9.4,10.0,9.4,9.7,9.4,0,0,0,0,0,0,1,42,2,10.0,4.214685e-08
1,115,9.7,9.5,9.8,9.8,9.7,9.6,9.7,1,0,0,0,0,0,1,48,2,13.5,1.550854e+00
2,117,9.7,9.4,9.3,9.7,9.5,9.7,9.4,0,0,0,0,0,0,0,8,2,17.0,3.351626e+00
3,116,10.0,10.0,10.0,10.0,10.0,10.0,9.7,0,0,0,0,0,0,0,41,2,18.0,3.705733e+00




Test Resuts for hostel.id=119

Similar hostels who are famous for excellent Value for money, Security, Staff, Atmosphere, Cleanliness, Facilities and Location.


,hostel.id,Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,Board.Games,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Distance,City,Price,similarity_distance
0,119,8.5,10.0,8.0,8.5,8.5,9.5,7.0,0,0,0,0,0,0,1,43,2,12.5,0.000000
1,118,9.4,9.7,9.4,10.0,9.4,9.7,9.4,0,0,0,0,0,0,1,42,2,10.0,5.086617
2,115,9.7,9.5,9.8,9.8,9.7,9.6,9.7,1,0,0,0,0,0,1,48,2,13.5,5.709043
3,117,9.7,9.4,9.3,9.7,9.5,9.7,9.4,0,0,0,0,0,0,0,8,2,17.0,5.927204




Test Resuts for hostel.id=120

Similar hostels who are famous for excellent Value for money, Security, Staff, Atmosphere, Cleanliness, Facilities and Location.


,hostel.id,Value.for.money,Security,Location,Staff,Atmosphere,Cleanliness,Facilities,Board.Games,Dvds,Foosball,Games.Room,PlayStation,Pool.Table,Wifi,Distance,City,Price,similarity_distance
0,120,9.0,8.0,10.0,10.0,9.0,10.0,8.0,0,0,0,0,0,0,0,6,2,23.75,0.000000
1,117,9.7,9.4,9.3,9.7,9.5,9.7,9.4,0,0,0,0,0,0,0,8,2,17.00,3.913075
2,116,10.0,10.0,10.0,10.0,10.0,10.0,9.7,0,0,0,0,0,0,0,41,2,18.00,5.039978
3,118,9.4,9.7,9.4,10.0,9.4,9.7,9.4,0,0,0,0,0,0,1,42,2,10.00,5.152377


The results look promising.